# Анализ данных Google Books

Ниже приведён код с подробными комментариями, поясняющими ключевые этапы анализа данных. Комментарии добавлены с расчётом на устную защиту проекта.

## Импорт библиотек

In [ ]:
# pandas — работа с таблицами данных
# numpy — численные вычисления
# matplotlib и seaborn — визуализация данных
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Загрузка данных

In [ ]:
# Загружаем CSV-файл.
# Предполагается, что файл находится в той же директории, что и ноутбук
df = pd.read_csv('google_books_dataset.csv')

# Просматриваем первые строки для понимания структуры данных
df.head()

## Первичный обзор данных

In [ ]:
# Последние строки датасета
df.tail()

# Общая информация: типы данных, количество непустых значений
df.info()

# Размер датасета (строки, столбцы)
df.shape

# Подсчёт пропущенных значений по каждому столбцу
df.isnull().sum()

## Обработка пропущенных значений

In [ ]:
# Удаляем столбцы с большим количеством пропусков и низкой аналитической ценностью
df = df.drop(columns=['subtitle', 'description'])

# Числовые признаки заполняем медианой — она устойчива к выбросам
df['average_rating'] = df['average_rating'].fillna(df['average_rating'].median())
df['page_count'] = df['page_count'].fillna(df['page_count'].median())

# Категориальные признаки заполняем модой (самым частым значением)
df['publisher'] = df['publisher'].fillna(df['publisher'].mode()[0])
df['categories'] = df['categories'].fillna(df['categories'].mode()[0])

## Дубликаты и выбросы

In [ ]:
# Удаляем полные дубликаты строк
df = df.drop_duplicates()

# Поиск выбросов по количеству страниц с помощью Z-score
# Значения с |z| > 3 считаются выбросами
z_scores = np.abs((df['page_count'] - df['page_count'].mean()) / df['page_count'].std())
outliers = df[z_scores > 3]

# Размер подтаблицы с выбросами
outliers.shape

In [ ]:
# Визуальная проверка выбросов с помощью boxplot
plt.boxplot(df['page_count'].dropna())
plt.title('Boxplot для количества страниц')
plt.ylabel('Page count')
plt.show()

## Приведение типов данных

In [ ]:
# Преобразуем дату публикации из строки в формат datetime
# errors='coerce' заменяет некорректные значения на NaT
df['published_date'] = pd.to_datetime(df['published_date'], errors='coerce')

# Проверяем типы данных после преобразования
df.dtypes

## Описательная статистика

In [ ]:
# Описательная статистика для числовых признаков
df.describe()

# Описательная статистика для категориальных признаков
df.describe(include='object')

## Уникальные значения и частоты

In [ ]:
# Количество уникальных значений в каждом столбце
df.nunique()

# Распределение книг по языкам
df['language'].value_counts()

# Топ-10 самых частых категорий
df['categories'].value_counts().head(10)

## Корреляционный анализ

In [ ]:
# Корреляционная матрица только для числовых признаков
corr_matrix = df.select_dtypes(include=np.number).corr()
corr_matrix

## Визуализация данных

In [ ]:
# Гистограмма распределения количества страниц
plt.hist(df['page_count'].dropna(), bins=30)
plt.title('Распределение количества страниц')
plt.xlabel('Page count')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Линейный график: средний рейтинг книг по годам
df['year'] = df['published_date'].dt.year
rating_by_year = df.groupby('year')['average_rating'].mean()

plt.plot(rating_by_year)
plt.title('Средний рейтинг книг по годам')
plt.xlabel('Год')
plt.ylabel('Средний рейтинг')
plt.show()

In [ ]:
# Boxplot для анализа распределения среднего рейтинга
plt.boxplot(df['average_rating'].dropna())
plt.title('Boxplot среднего рейтинга')
plt.ylabel('Average rating')
plt.show()

In [ ]:
# Heatmap корреляций для наглядного анализа связей между признаками
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Heatmap корреляций')
plt.show()

## Группировки и агрегирование

In [ ]:
# Средний рейтинг по категориям
df.groupby('categories')['average_rating'].mean().head(10)

# Медианное количество страниц по категориям
df.groupby('categories')['page_count'].median().head(10)

# Количество книг по языкам
df.groupby('language')['book_id'].count()

# Суммарное количество страниц по категориям
df.groupby('categories')['page_count'].sum().head(10)

## Сводная таблица (Pivot Table)

In [ ]:
# В столбце categories содержится несколько категорий в одной строке.
# Для корректного анализа выделяем основную (первую) категорию
df['main_category'] = df['categories'].str.split('/').str[0].str.strip()

# Оставляем только самые популярные категории
top_main = df['main_category'].value_counts().head(8).index

# Строим сводную таблицу: средний рейтинг по языкам и категориям
pivot_clean = pd.pivot_table(
    df[df['main_category'].isin(top_main)],
    values='average_rating',
    index='language',
    columns='main_category',
    aggfunc='mean'
)
pivot_clean